In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 1000, stop = 1500, num = 3)]
max_features = [.2, 5]#, 'log2']  # Number of features to consider at every split
max_depth = [2,4]#[int(x) for x in np.linspace(2, 5, num = 3)]   # Maximum number of levels in tree
#max_depth.append(None)
min_samples_split = [2, 5, 10]  # Minimum number of samples required to split a node
min_samples_leaf = [1, 10, 15]    # Minimum number of samples required at each leaf node
bootstrap = [True]                # Method of selecting samples for training each tree
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_leaf_nodes': [None] + list(np.linspace(15, 60, 4).astype(int)),
               'bootstrap': bootstrap}

rf = RandomForestClassifier(oob_score=True, n_jobs=-1)
rf_random = RandomizedSearchCV(
                estimator = rf,
                param_distributions = random_grid,
                n_iter = 150, cv = 3,
                verbose=1, random_state=22,
                scoring='accuracy')

In [ ]:
pipe_random = make_pipeline(col_trans, rf_random)
pipe_random.fit(train_x, train_y)
rf_random.best_params_

In [ ]:

best_params_lst = []
pipe_best_model_lst = []
top_five = ['NAmes', 'CollgCr']#, 'OldTown', 'Edwards', 'Somerst']
top_five_dicts = []
for name in top_five:
    print(name)
    name_ = train_xy.loc[train_xy['Neighborhood'] == name].reset_index(drop=True)
    t_x = name_.iloc[:,1:]
    t_y = name_.iloc[:,-1:]
    col_trans = make_column_transformer(
                        (OneHotEncoder(),t_x.columns.tolist()),
                        remainder = "passthrough"
                        )
    pipe_random = make_pipeline(col_trans, rf_random)
    pipe_random.fit(t_x, t_y)
    rf_random.best_params_
    best_params_lst.append(rf_random.best_params_)
    pipe_best_model_lst.append(pipe_random)

In [ ]:
pipe_best_model_lst[0].fit(t_x,t_y)

In [ ]:
# To look at nodes and depths of trees use on average
n_nodes = []
max_depths = []
est_ = []



best_model = rf_random.best_estimator_
pipe_best_model = make_pipeline(col_trans, best_model)
pipe_best_model.fit(train_x, train_y)
#y_pred_best_model = pipe_best_model.predict(train_x)

In [ ]:
# how to reshape the data to fit
np.array(t_y)[:, None].reshape(-1,1)

In [ ]:
pipe_best_model.score(t_x,(np.array(t_y)[:, None].reshape(-1,1)))#train_x, train_y)
#pipe_best_model.oob_score(train_x, train_y)
#ind_tree.tree_.value.shape

In [ ]:
for ind_tree in best_model.estimators_:
       n_nodes.append(ind_tree.tree_.node_count)
       max_depths.append(ind_tree.tree_.max_depth)
       est_.append(ind_tree.score)
       
print(f'Average number of nodes {int(np.mean(n_nodes))}')   
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
X_train_encoded_ = encode_and_bind(t_x, t_x.columns.tolist())

In [ ]:
export_graphviz(est_limited, out_file='tree.dot', feature_names= X_train_encoded_,#X_train_encoded.columns,
                class_names = t_y, rounded = True, proportion = False,
                precision = 2, filled = True)

from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', 'Gdpi=600'])

from IPython.display import Image
Image(filename = 'tree.png')

In [ ]:
for i, v in enumerate(test_x.columns):
    print(i, v)

In [ ]:
train_rf_predictions = pipe_best_model.predict(train_x)
train_rf_probs = pipe_best_model.predict_proba(train_x)[:, 1]

In [ ]:
print(len(train_rf_predictions))
for i, v in enumerate(train_rf_predictions):
    print(v, train_y[i], train_rf_probs[i])

In [ ]:
df_results = pd.DataFrame([train_rf_predictions, train_y[::1], train_rf_probs])

In [ ]:
train_rf_predictions
train_rf_probs

In [ ]:
train_y[::1]

In [ ]:
df_results = df_results.transpose()

In [ ]:
df_results.columns = ['Predicted', 'Actual', 'Prob_a']

In [ ]:
def evaluate_model(y_pred, probs,train_predictions, train_probs):
    baseline = {}
    baseline['recall']=recall_score(y_test,
                    [1 for _ in range(len(y_test))])
    baseline['precision'] = precision_score(y_test,
                    [1 for _ in range(len(y_test))])
    baseline['roc'] = 0.5
    results = {}
    results['recall'] = recall_score(y_test, y_pred)
    results['precision'] = precision_score(y_test, y_pred)
    results['roc'] = roc_auc_score(y_test, probs)
    train_results = {}
    train_results['recall'] = recall_score(y_train,       train_predictions)
    train_results['precision'] = precision_score(y_train, train_predictions)
    train_results['roc'] = roc_auc_score(y_train, train_probs)
    for metric in ['recall', 'precision', 'roc']:  
          print(f'{metric.capitalize()} \
                 Baseline: {round(baseline[metric], 2)} \
                 Test: {round(results[metric], 2)} \
                 Train: {round(train_results[metric], 2)}')
     # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(y_test, [1 for _ in range(len(y_test))])
    model_fpr, model_tpr, _ = roc_curve(y_test, probs)
    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate');
    plt.ylabel('True Positive Rate'); plt.title('ROC Curves');
    plt.show();
#evaluate_model(y_pred_best_model,rf_probs,train_rf_predictions,train_rf_probs)